# Chapter 18. With, match, and else blocks

Example 18-1. Demonstration of a file object as a context manager

In [2]:
with open('mirror.py') as fp:
    src = fp.read(60)

In [3]:
src

''

In [4]:
len(src)

0

In [5]:
fp

<_io.TextIOWrapper name='mirror.py' mode='r' encoding='UTF-8'>

In [6]:
fp.closed, fp.encoding

(True, 'UTF-8')

In [7]:
fp.read(60)

ValueError: I/O operation on closed file.

Example 18-2. Test-driving the LookingGlass context manager class

In [1]:
from mirror import LookingGlass

In [9]:
with LookingGlass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [10]:
what

'JABBERWOCKY'

In [11]:
print('back to normal')

back to normal


Example 18-4. Exercising LookingGlass without a with block

In [16]:
manager = LookingGlass()

In [17]:
manager

In [23]:
monster = manager.__enter__()

In [24]:
monster

'JABBERWOCKY'

In [25]:
monster == 'JABBERWOCKY'

True

In [26]:
manager.__exit__(None, None, None)

In [27]:
monster

'JABBERWOCKY'

Example 18-6. Test-driving the looking_glass context manager function

In [2]:
from mirror import looking_glass

with looking_glass() as what:
    print('Jalil va Paria ichdi buyun su')
    print(what)

us nuyub idhci airaP av lilaJ
YKCOWREBBAJ


In [3]:
what

'JABBERWOCKY'

In [4]:
@looking_glass()
def verse():
    print('Alivi ver mana')

In [5]:
verse()

anam rev ivilA


In [6]:
print('bir iki')

bir iki


Example 18-10. Greatest common divisor in Scheme

In [2]:
def mod(m, n):
    return m - (m // n * n)

def gcd(m, n):
    if n == 0:
        return m
    else:
        return gcd(n, mod(m, n))

print(gcd(18, 45))

9


Example 18-14. lis.py: the Environment class

In [20]:
from collections import ChainMap
import operator as op
from itertools import chain
from typing import Any, TypeAlias, NoReturn
import math

In [34]:
Symbol: TypeAlias = str
Atom: TypeAlias = float | int | Symbol
Expression: TypeAlias = Atom | list

In [31]:
class Environment(ChainMap[Symbol, Any]):
    "A ChainMap that allows changing an item in-place."
    def change(self, key: Symbol, value: Any) -> None:
        "Find where key is defined and change the value there."
        for map in self.maps:
            if key in map:
                map[key] = value
                return
        raise KeyError(key)

In [32]:
inner_env = {'a': 2}
outer_env = {'a': 0, 'b': 1}

env = Environment(inner_env, outer_env)

env['a']

2

In [43]:
env['a'] = None
env

Environment({'a': None}, {'a': 0, 'b': 1})

In [42]:
env['a'] = 111
env

Environment({'a': 111}, {'a': 0, 'b': 1})

In [18]:
env.change('b', 342)

In [19]:
env

Environment({'a': 111}, {'a': 0, 'b': 342})

In [24]:
def standard_env() -> Environment:
    env = Environment()
    env.update(vars(math))
    env.update({
        '+': op.add,
        '-': op.sub,
        '/': op.truediv,
        '*': op.mul,
        'abs': abs,
        'append': lambda *args: list(chain(*args)),
        'apply': lambda proc, args: proc(*args),
        'begin': lambda *x: x[-1],
        'car': lambda x: x[1:],
        'number?': lambda x: isinstance(x, (int, float)),
        'procedure?': callable,
        'round': round,
        'symbol?': lambda x: isinstance(x, Symbol),        
    })
    return env

In [37]:
KEYWORDS = ['quote', 'if', 'lambda', 'define', 'set!']

def evaluate(exp: Expression, env: Environment) -> Any:
    "Evaluate an expression in an environment."
    match exp:
        case int(x) | float(x):
            return x
        case Symbol(var):
            return env[var]
        case ['quote', x]:
            return x
        case ['if', test, consequence, alternative]:
            if evaluate(test, env):
                return evaluate(consequence, env)
            else:
                return evaluate(alternative, env)
        case _:
            print('No return')
        

In [39]:
evaluate(['quote', 5],{})

5

In [40]:
evaluate(['@', 5],{})

No return
